Dans ce fichier, on définit plusieurs fonctions et méthodes relatifs aux traitements d'un noeud de l'arbre du Branch-and-Price.

On commence par définir le problème maitre (formulation présentée dans le main) :

In [42]:
function master(noeud_col)
    # Initialisation du probleme maitre
    master = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(master, "OutputFlag", 0)
    
    # Definition de la variable z
    # L'indice de z correspond a l'indice du cycle
    z = Array{VariableRef,1}(undef,size(noeud_col,1))
    for c in 1:size(noeud_col,1)
        z[c] = @variable(master,lower_bound=0)
        set_name(z[c],"z_$c")
    end
    
    # Fonction objective
    @objective(master,Max,sum(noeud_col[c][1]*z[c] for c in 1:size(noeud_col,1)))
    
    # Contrainte : Chaque paire participe à au plus un cycle
    
    # Si une paire est dans un cyle, alors le cycle contient un arc dont la paire est à l'origine
    # definition de  y_c_i = 1 si sommet i dans le cycle c, 0 sinon
    y = zeros(Int, size(noeud_col,1), nv(pool))
    for c in 1:size(noeud_col,1)
        for v in 1:nv(pool)
            if(!isempty(origine[v]))
                y[c,v] = sum(noeud_col[c][k+1] for k in origine[v])
            end
        end
    end
    
    paire = Array{ConstraintRef,1}(undef,nv(pool))
    for v in 1:nv(pool)
        paire[v] = @constraint(master,sum(y[c,v]*z[c] for c in 1:size(noeud_col,1))<=1)
        set_name(paire[v],"paire_$v")
    end
    
    # On retourne le modèle avec ses variables et contraintes
    return paire,z,master
end

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[42]:9

On écrit une fonction qui permet de traiter le sous-problème associé au master.

In [12]:
function subproblem(π,node)
    SP=Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(SP, "OutputFlag", 0)
    
    # Definition x_a = 1 si l'arc a est dans le cycle retenu, 0 sinon
    @variable(SP,x[a in 1:ne(pool)],Bin)
    
    # Definition de y_i = 1 si paire i dans le cycle retenu, 0 sinon
    @variable(SP,y[i in 1:nv(pool)],Bin)
    
    
    for vertice in 1:nv(pool)
        # Contraintes : les arcs doivent former un cycle
        @constraint(SP,sum(x[a] for a in origine[vertice]) == sum(x[a] for a in dest[vertice]) )
        #Contrainte : y_i vaut 1 ssi il y a un arc qui sort de la paire i qui est choisi dans le cycle
        #@constraint(SP, y[vertice] == sum(x[a] for a in origine[vertice]))
        @constraint(SP,sum(x[a] for a in origine[vertice])<=1)
    end
    
    # Contrainte : le cycle doit être de longueur inférieure à L 
    @constraint(SP, sum(x[a] for a in 1:ne(pool)) <= L) 
    
    # On veut un cycle, donc on doit avoir au moins deux sommets
    # @constraint(SP, sum(y[i] for i in 1:nv(pool)) >= 2)
    
    # Objectif : maximiser le coût réduit
    @objective(SP,Max,sum( weights[a]*x[a] for a in 1:ne(pool) ) - sum(π[i]*x[a] for i in 1:nv(pool),a in origine[i] ))
    
    # Contraintes permettant de refuser ou d'imposer certains arcs dans la solution
    for arc in tree[node].setzero
        @constraint(SP,x[arc]==0)
    end

    for arc in tree[node].setone
        @constraint(SP,x[arc]==1)
    end

    optimize!(SP)

    # If the subproblem is solved to optimality then return its objective value
    # and the optimal cycle solution
    # Else the pricing problem is infeasible return Inf, []
    if JuMP.termination_status(SP)==MOI.OPTIMAL
        return JuMP.objective_value(SP),JuMP.value.(x)
    else
        return -Inf,[]
    end
end

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[12]:6

In [1]:
function Process_Node(nodeindex)
    println("\e[93m Processing node $nodeindex \e[00m")
    println("Variables set to zero are:", tree[nodeindex].setzero)
    println("Variables set to one are:", tree[nodeindex].setone)
    global nodestobedeleted = []
    
    # Calcul des colonnes compatibles avec le branchement
    noeud_col = calculate_columns(nodeindex)
    
    # On recupère les données de la relaxation du maitre restreint :
    paire,z,mastermodel = master(noeud_col)
    
    while true
        # Resolution du maitre restreint
        optimize!(mastermodel)
        #print("\nA quoi ressemble l'ensemble des noeuds ?\n\n $(noeud_col)\n\n")
        node_infeasible = false
        
        # On verifie si le problème est réalisable
        if termination_status(mastermodel) == MOI.INFEASIBLE 
            println("\e[31m Noeud non réalisable \e[00m")
            return []
        end
        
        #print("Valeur contraintes : $(JuMP.value.(paire))\n\n")
        
        #print("VALEUR SOLUTION z : $(JuMP.value.(z))\n\n")
        #print("POIDS SOLUTION z : $(JuMP.objective_value(mastermodel))\n\n")
        # Recupèration de la solution duale pour les contraintes
        π = -JuMP.dual.(paire)
        #print("Valeur du dual $(π)\n")
        # Recupération de la valeur de la fonc objective
        value = JuMP.objective_value(mastermodel)
        # Extraction de la solution
        
        solution = Array{Float32,1}(undef,size(noeud_col,1))
        for cycle in 1:size(noeud_col,1)
            solution[cycle]=JuMP.value.(z)[cycle]
        end
        solflat = collect(Iterators.flatten(solution))
        
        # On verifie si la solution est entière
        # Si c'est le cas on met à jour la borne duale
        if maximum(solflat-floor.(solflat)) <= ϵ
            # Une solution realisable est trouvée
            if value>=LB global LB=value end
            for i in 1:length(Queue)-1
                if tree[Queue[i]].ub <= LB
                    push!(nodestobedeleted,i)
                end
            end
            println("\e[32m Solution réalisable de valeur $value trouvée \e[00m")
        end
        
        # Mise à jour des bornes primales et duales
        nodelb = value
        nodeub = sum(π[i] for i in 1:nv(pool))
        
        minobj = 0
        
        # Resolution du sous probleme
        # recuperation cycle optimal et valeur optimale
        SPobj,sol = subproblem(π,nodeindex)
        
        for c in 1:size(sol,1)
            for a in size(sol[c],1)
                sol[a] = round(Int32,sol[a]) # permet d'arrondir, column sortait parfois des 
                                               #valeurs non égale à 0 ou 1 mais très proche de cela
            end
        end
        
        if SPobj>-Inf
            # On recupère les cycles créés par résolution du sous probleme
            cycles = calcul_cycles(sol)
            # Calcul du coût réduit
            cout_reduit = SPobj
            #print("COUT REDUIT : $(cout_reduit)\n")
            
            if cout_reduit>=minobj minobj = cout_reduit end
            #print("MIN OBJ : $(minobj)\n")
            
            # Si on trouve un cout reduit positif (car maximisation), on ajoute les cycles au master
            if cout_reduit>0
                for column in cycles
                    # Calcul du cout de la colonne trouvee
                    col_cout = sum(weights[arc]*column[arc] for arc in 1:ne(pool))

                    col_cout = round(Int64, col_cout)
                    # On ajoute la colonne dans l'ensemble des colonnes
                    push!(column_pool,vcat(col_cout,column))
                    # On ajoute la colonne dans l'ensemble des colonnes du noeud
                    push!(noeud_col,vcat(col_cout,column))

                    # On ajoute la nouvelle colonne dans le modele
                    push!(z,@variable(mastermodel,lower_bound=0))
                    #Nom de la nouvelle variable
                    set_name(z[end], "z_$(size(column_pool,1))")

                    # On ajoute la colonne dans les contraintes et dans la fonction objective
                    for i in 1:nv(pool)
                        if(!isempty(origine[i]))
                            set_normalized_coefficient(paire[i], z[end],sum(column[j] for j in origine[i]))
                        else
                            set_normalized_coefficient(paire[i], z[end],0)
                        end
                    end
                    set_objective_function(mastermodel, objective_function(mastermodel) + col_cout*z[end])
                    cycle = findall(x->x!=0,column)
                    println("\e[34m Cycle avec les arcs $cycle ajouté \e[00m")
                end
            end
            # Complete the lower bound calculation
            nodeub += SPobj
        else
            node_infeasible = true
            println("\e[31m Noeud non réalisable \e[00m")
            break
        end
        
        # Si le sous probleme est infaisable on  ne retourne rien
        if node_infeasible return [] end

        # On UPDATE si on améliore la borne supérieure
        #if nodeub<=tree[nodeindex].ub 
        tree[nodeindex].ub = nodeub 
        #end
        
        # Si l'algorithme converge on STOP
        if 2*abs((nodelb-nodeub))/abs((nodelb+nodeub))<ϵ || minobj<=ϵ
            println("Node relaxation is solved to optimality")
            println("Node upper bound is $(tree[nodeindex].ub)")
            println("Node lower bound is $nodelb")
            # Si la colonne artificielle est utilisée dans la solution
            # alors le probleme maitre est non realisable
            if solution[1]>=ϵ
                node_infeasible = true
                println("\e[31m Node is infeasible \e[00m")
            end
            # Si non realisable RETURN EMPTY
            if node_infeasible  return []
            # Si realisable RETURN SOLUTION
            else return calculate_xsol(solution,noeud_col) end
        end
    end

end

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[1]:91

-0.0